# source

> Code that emits markdown from the `source` element of various notebook cell types

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp source

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| hide
#| export
import io
from typing import Dict, Iterable, Optional, Sequence, Tuple

In [ ]:
#| export
from beetroot.transformations import emit_with_transformation, Transformer

In [ ]:
# | exporti
def emit_markdown_source(markdown: Iterable[str], stream: io.TextIOBase):
    for line in markdown:
        stream.write(line)
    stream.write('\n')

In [ ]:
# | exporti


def is_directive_line(line: str):
    return line.startswith('#|') or line.startswith('# |')


def parse_directive_line(line: str) -> Tuple[str, Optional[bool]]:
    assert is_directive_line(line)

    directive = line.lstrip('# |').strip()
    parts = [part.strip() for part in directive.split(':')]

    # A directive is either a single key or key: value
    assert len(parts) == 1 or len(parts) == 2
    key, *value_list = parts
    value_str: Optional[str] = value_list[0] if value_list else None

    # Deal with string forms of true and false
    # These directives are technically YAML, so allowing all the values
    # for bool per [the spec](https://yaml.org/type/bool.html).

    true_vals = [
        'y',
        'Y',
        'yes',
        'Yes',
        'YES',
        'true',
        'True',
        'TRUE',
        'on',
        'On',
        'ON',
    ]
    false_vals = [
        'n',
        'N',
        'no',
        'No',
        'NO',
        'false',
        'False',
        'FALSE',
        'off',
        'Off',
        'OFF',
    ]

    value: Optional[bool] = None
    if value_str in true_vals:
        value = True
    elif value_str in false_vals:
        value = False

    return key, value

In [ ]:
#| hide 
test_cases = [
    {
        'line': '#| hide',
        'expected': ('hide', None)
    },
    {
        'line': '#| echo: true',
        'expected': ('echo', True)
    },
    {
        'line': '# | echo: false',
        'expected': ('echo', False)
    },
    {
        'line': '# | code-fold: On',
        'expected': ('code-fold', True)
    },
]

for tc in test_cases:
    actual = parse_directive_line(tc['line'])
    test_eq(actual, tc['expected'])

In [ ]:
# | exporti
def parse_and_extract_directives_from_python_source(
    source: Sequence[str],
) -> Tuple[Sequence[str], Dict[str, Optional[bool]]]:
    directives = {}
    i = 0  # initialize explicitly because `source` may be empty
    for i, line in enumerate(source):
        if is_directive_line(line):
            key, value = parse_directive_line(line)
            directives[key] = value
        else:
            # Break out on hitting the first non-directive line
            break

    return source[i:], directives

In [ ]:
# | hide
# Test extraction of directives

test_cases = [
    {
        'name': 'Basic',
        'python_source': [
            '# | echo: false',
            '# | output: false',
            'name = \'world\'\n',
            'print(f"hello, {name}")',
        ],
        'expected_rest_of_source': ['name = \'world\'\n', 'print(f"hello, {name}")'],
        'expected_directives': {
            'echo': False,
            'output': False,
        },
    },
    {
        'name': 'Mix of # | and #|',
        'python_source': [
            '# | echo: false',
            '#| output: false',
            'name = \'world\'\n',
            'print(f"hello, {name}")',
        ],
        'expected_rest_of_source': ['name = \'world\'\n', 'print(f"hello, {name}")'],
        'expected_directives': {
            'echo': False,
            'output': False,
        },
    },
    {
        'name': 'No directives',
        'python_source': [
            'name = \'world\'\n',
            'print(f"hello, {name}")',
        ],
        'expected_rest_of_source': ['name = \'world\'\n', 'print(f"hello, {name}")'],
        'expected_directives': {},
    },
    {
        'name': 'Directive after first non-directive line is ignored',
        'python_source': [
            '# | echo: false',
            '#| output: false',
            'name = \'world\'\n',
            '# | ignore: true\n',
            'print(f"hello, {name}")',
        ],
        'expected_rest_of_source': [
            'name = \'world\'\n',
            '# | ignore: true\n',
            'print(f"hello, {name}")',
        ],
        'expected_directives': {
            'echo': False,
            'output': False,
        },
    },
]

for tc in test_cases:
    name = tc['name']
    python_source = tc['python_source']
    expected_rest_of_source = tc['expected_rest_of_source']
    expected_directives = tc['expected_directives']

    print(f"Running test case: {name}")
    rest_of_source, directives = parse_and_extract_directives_from_python_source(
        python_source
    )

    test_eq(rest_of_source, expected_rest_of_source)
    test_eq(directives, expected_directives)

Running test case: Basic
Running test case: Mix of # | and #|
Running test case: No directives
Running test case: Directive after first non-directive line is ignored


In [ ]:
# | exporti
def emit_python_source(source: Sequence[str], stream: io.TextIOBase):
    stream.write('```python\n')
    for line in source:
        stream.write(line)
    stream.write('\n```\n\n')


In [ ]:
# | export
class SourceHandler:
    """High-level API for emitting cell source"""
    def __init__(
        self,
        stream: io.TextIOBase,
        transformers_map: Dict[str, Transformer] = {},
    ):
        self.stream = stream
        self.transformers_map = transformers_map

    def emit_markdown(self, lines: Sequence[str]):
        emit_with_transformation(
            self.transformers_map.get('markdown/source', Transformer()),
            lines,
            emit_markdown_source,
            self.stream,
        )

    def emit_python_source(self, lines: Sequence[str]) -> Tuple[bool, bool]:
        python_source, directives = parse_and_extract_directives_from_python_source(
            lines
        )

        # Handle directives per https://quarto.org/docs/reference/cells/cells-jupyter.html#code-output
        # and https://quarto.org/docs/reference/cells/cells-jupyter.html#cell-output.
        should_echo = 'echo' not in directives or directives['echo'] == True
        should_show_output = 'output' not in directives or directives['output'] == True

        if should_echo:
            emit_with_transformation(
                self.transformers_map.get('python/source', Transformer()),
                python_source,
                emit_python_source,
                self.stream,
            )

        return should_echo, should_show_output

In [ ]:
show_doc(SourceHandler.emit_markdown)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/source.py#L109){target="_blank" style="float:right; font-size:smaller"}

### SourceHandler.emit_markdown

>      SourceHandler.emit_markdown (lines:Sequence[str])

In [ ]:
markdown_source = [
    '# This is Markdown\n',
    'This is a [link](https://www.google.com).\n',
    'This is:\n',
    '\n',
    '* a \n',
    '* bulleted\n',
    '* list\n',
    '\n',
    'Yup.',
]
stream = io.StringIO()
src_handler = SourceHandler(stream)

src_handler.emit_markdown(markdown_source)

stream.seek(0)
output = stream.read()

expected = """\
# This is Markdown
This is a [link](https://www.google.com).
This is:

* a 
* bulleted
* list

Yup.
"""

test_eq(output, expected)
print(output)

# This is Markdown
This is a [link](https://www.google.com).
This is:

* a 
* bulleted
* list

Yup.



In [ ]:
show_doc(SourceHandler.emit_python_source)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/source.py#L117){target="_blank" style="float:right; font-size:smaller"}

### SourceHandler.emit_python_source

>      SourceHandler.emit_python_source (lines:Sequence[str])

In [ ]:
python_source = ['name = \'world\'\n', 'print(f"hello, {name}")']

stream = io.StringIO()
src_handler = SourceHandler(stream)

did_echo, should_show_output = src_handler.emit_python_source(python_source)

stream.seek(0)
output = stream.read()

expected = """\
```python
name = 'world'
print(f"hello, {name}")
```

"""

test_eq(output, expected)
test_eq(did_echo, True)
test_eq(should_show_output, True)
print(output)

```python
name = 'world'
print(f"hello, {name}")
```




In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()